In [1]:
import MetaTrader5 as mt5 # to access historcal data
import pandas as pd # for data analysis and calculation of technical indcator
import numpy as np
import tdclient as TDClient
import plotly.express as px # for data visualization
import plotly.graph_objects as go
from datetime import datetime, time ,timezone #to specify the date time range for historical data
from IPython.display import display, Markdown, Latex # to display result in python notebook

In [2]:
# Conecte mt5
mt5.initialize()
# logine mt5
login = 1520462586
password = 'vs6X$@@*B'
server = 'FTMO-Demo2'

mt5.login(login,password,server)

False

In [14]:
# this function retreives olhc data from mt5 account and return a data frame
def get_ohlc(symbol, timeframe, start_datetime, end_datetime):
    ohlc = mt5.copy_rates_range(symbol, timeframe, start_datetime, end_datetime)
    ohlc_df = pd.DataFrame(ohlc)
    ohlc_df['time'] = pd.to_datetime(ohlc_df['time'], unit= 's')
    return ohlc_df[['time', 'open', 'high', 'low', 'close']]

In [20]:
# 30 minut df
symbol = 'EURUSD'
pos_size = 1
timeframe = mt5.TIMEFRAME_M30
start_datetime = datetime(2024,1,1)
end_datetime = datetime(2024,4,19)
ohlc_df = get_ohlc(symbol, timeframe, start_datetime, end_datetime)
ohlc_df

KeyError: 'time'

In [19]:
# hour4 df
symbol = 'EURUSD'
pos_size = 1
timeframe = mt5.TIMEFRAME_H4
start_datetime = datetime(2024,1,1)
end_datetime = datetime(2024,4,19)
hourly_df = get_ohlc(symbol, timeframe, start_datetime, end_datetime)
hourly_df

KeyError: 'time'

In [57]:
# daily df
symbol = 'EURUSD'
pos_size = 1
timeframe = mt5.TIMEFRAME_D1
start_datetime = datetime(2024,1,1)
end_datetime = datetime(2024,4,19)
daily_df = get_ohlc(symbol, timeframe, start_datetime, end_datetime)
daily_df

,time,open,high,low,close
0,2024-01-02,1.10424,1.10443,1.09383,1.09413
1,2024-01-03,1.09423,1.09654,1.08929,1.09196
2,2024-01-04,1.09163,1.09721,1.09140,1.09478
3,2024-01-05,1.09442,1.09984,1.08765,1.09409
4,2024-01-08,1.09411,1.09789,1.09224,1.09508
...,...,...,...,...,...
73,2024-04-12,1.07271,1.07293,1.06225,1.06413
74,2024-04-15,1.06342,1.06650,1.06202,1.06225
75,2024-04-16,1.06236,1.06536,1.06011,1.06184
76,2024-04-17,1.06179,1.06796,1.06060,1.06733


In [49]:
# start b2 strategy
# Calculate Pivot Point, R1, R2, R3, stopeR4, S1, S2, S3, stopeS4 levels with Fibonacci retracement
daily_df["pivot"] = (daily_df["high"] + daily_df["low"] + daily_df["close"]) / 3
daily_df["prange"] = (daily_df["high"] - daily_df["low"])
daily_df["R1"] = daily_df["pivot"] + 0.382 * daily_df["prange"]
daily_df["R2"] = daily_df["pivot"] + 0.618 * daily_df["prange"]
daily_df["R3"] = daily_df["pivot"] + 1.000 * daily_df["prange"]

daily_df["S1"] = daily_df["pivot"] - 0.382 * daily_df["prange"]
daily_df["S2"] = daily_df["pivot"] - 0.618 * daily_df["prange"]
daily_df["S3"] = daily_df["pivot"] - 1.000 * daily_df["prange"]
daily_df

,time,open,high,low,close,pivot,prange,R1,R2,R3,...,sma_11,prev_sma_11,sma_114,stopeR4,bearish_pivot,stopeS4,bullish_pivot,profitR5,profitS5,entring_stoploss_profit
0,2024-01-02,1.10424,1.10443,1.09383,1.09413,1.097463,0.01060,1.101513,1.104014,1.108063,...,1.09413,NaN,NaN,False,True,False,False,False,True,False
1,2024-01-03,1.09423,1.09654,1.08929,1.09196,1.092597,0.00725,1.095366,1.097077,1.099847,...,1.09196,1.09413,NaN,True,True,False,False,False,True,False
2,2024-01-04,1.09163,1.09721,1.09140,1.09478,1.094463,0.00581,1.096683,1.098054,1.100273,...,1.09478,1.09196,NaN,True,True,False,False,False,True,False
3,2024-01-05,1.09442,1.09984,1.08765,1.09409,1.093860,0.01219,1.098517,1.101393,1.106050,...,1.09409,1.09478,NaN,False,True,False,False,False,True,False
4,2024-01-08,1.09411,1.09789,1.09224,1.09508,1.095070,0.00565,1.097228,1.098562,1.100720,...,1.09508,1.09409,NaN,True,True,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73,2024-04-12,1.07271,1.07293,1.06225,1.06413,1.066437,0.01068,1.070516,1.073037,1.077117,...,1.06413,1.07258,1.079191,True,True,False,False,False,True,False
74,2024-04-15,1.06342,1.06650,1.06202,1.06225,1.063590,0.00448,1.065301,1.066359,1.068070,...,1.06225,1.06413,1.077699,True,True,False,False,False,True,False
75,2024-04-16,1.06236,1.06536,1.06011,1.06184,1.062437,0.00525,1.064442,1.065681,1.067687,...,1.06184,1.06225,1.076203,True,True,False,False,False,True,False
76,2024-04-17,1.06179,1.06796,1.06060,1.06733,1.065297,0.00736,1.068108,1.069845,1.072657,...,1.06733,1.06184,1.075392,True,True,False,False,False,True,False


In [37]:
#setting the sma 1 and 14 periods
furst_sma_period = 1
second_sma_period = 14
# sma 1
ohlc_df['sma_1'] = hourly_df['close'].rolling(furst_sma_period).mean()
ohlc_df['prev_sma_1'] = ohlc_df['sma_1'].shift(1) #to find crossovers, previous sma value is necessary using shift()
# sma 14
ohlc_df['sma_14'] = hourly_df['close'].rolling(second_sma_period).mean()
ohlc_df

,time,open,high,low,close,R1,R2,R3,S1,S2,S3,sma_1,prev_sma_1,sma_14
0,2024-01-02 00:00:00,1.10424,1.10434,1.10416,1.10430,1.101513,1.104014,1.108063,1.093414,1.090913,1.086863,1.10349,NaN,NaN
1,2024-01-02 00:30:00,1.10422,1.10443,1.10416,1.10434,1.095366,1.097077,1.099847,1.089827,1.088116,1.085347,1.10334,1.10349,NaN
2,2024-01-02 01:00:00,1.10438,1.10442,1.10369,1.10375,1.096683,1.098054,1.100273,1.092244,1.090873,1.088653,1.10152,1.10334,NaN
3,2024-01-02 01:30:00,1.10375,1.10384,1.10355,1.10364,1.098517,1.101393,1.106050,1.089203,1.086327,1.081670,1.09527,1.10152,NaN
4,2024-01-02 02:00:00,1.10366,1.10380,1.10341,1.10345,1.097228,1.098562,1.100720,1.092912,1.091578,1.089420,1.09506,1.09527,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3736,2024-04-18 21:00:00,1.06481,1.06493,1.06436,1.06437,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3737,2024-04-18 21:30:00,1.06437,1.06485,1.06419,1.06451,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3738,2024-04-18 22:00:00,1.06450,1.06482,1.06424,1.06433,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3739,2024-04-18 22:30:00,1.06432,1.06468,1.06431,1.06437,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
#setting the 11 and 114 sma periods
thurd_sma_period = 1
fourth_sma_period = 14
# sma 11
ohlc_df['sma_11'] = daily_df['close'].rolling(thurd_sma_period).mean()
ohlc_df['prev_sma_11'] = ohlc_df['sma_11'].shift(1) #to find crossovers, previous sma value is necessary using shift()
# sma 114
ohlc_df['sma_114'] = daily_df['close'].rolling(fourth_sma_period).mean()
ohlc_df

,time,open,high,low,close,R1,R2,R3,S1,S2,S3,sma_1,prev_sma_1,sma_14,sma_11,prev_sma_11,sma_114
0,2024-01-02 00:00:00,1.10424,1.10434,1.10416,1.10430,1.101513,1.104014,1.108063,1.093414,1.090913,1.086863,1.10349,NaN,NaN,1.09413,NaN,NaN
1,2024-01-02 00:30:00,1.10422,1.10443,1.10416,1.10434,1.095366,1.097077,1.099847,1.089827,1.088116,1.085347,1.10334,1.10349,NaN,1.09196,1.09413,NaN
2,2024-01-02 01:00:00,1.10438,1.10442,1.10369,1.10375,1.096683,1.098054,1.100273,1.092244,1.090873,1.088653,1.10152,1.10334,NaN,1.09478,1.09196,NaN
3,2024-01-02 01:30:00,1.10375,1.10384,1.10355,1.10364,1.098517,1.101393,1.106050,1.089203,1.086327,1.081670,1.09527,1.10152,NaN,1.09409,1.09478,NaN
4,2024-01-02 02:00:00,1.10366,1.10380,1.10341,1.10345,1.097228,1.098562,1.100720,1.092912,1.091578,1.089420,1.09506,1.09527,NaN,1.09508,1.09409,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3736,2024-04-18 21:00:00,1.06481,1.06493,1.06436,1.06437,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3737,2024-04-18 21:30:00,1.06437,1.06485,1.06419,1.06451,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3738,2024-04-18 22:00:00,1.06450,1.06482,1.06424,1.06433,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3739,2024-04-18 22:30:00,1.06432,1.06468,1.06431,1.06437,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [50]:
#setting entring and closing time and one trade a day
# Set the start and end times for your desired trading hours
start_time = datetime.strptime('06:00:00', '%H:%M:%S').time()
end_time = datetime.strptime('15:00:00', '%H:%M:%S').time()

# Filter the trade data for trades within the specified trading hours
trades_within_hours = daily_df[(daily_df['time'].dt.time >= start_time) & (daily_df['time'].dt.time <= end_time)]

# Close all open positions at 11:45 PM
first_trade_per_day = daily_df.groupby(daily_df['time'].dt.time).first()
end_trading_time = datetime.strptime('23:45:00', '%H:%M:%S').time()
open_positions = trades_within_hours[trades_within_hours['time'].dt.time == end_trading_time]
# function to trading
daily_df["stopeR4"] = daily_df["high"] >= daily_df["R2"] + 0.0001*40
daily_df["bearish_pivot"] = daily_df["high"] >= daily_df["R2"]
daily_df["stopeS4"] = daily_df["low"] <= daily_df["S2"] - 0.0001*40
daily_df["bullish_pivot"] = daily_df["low"] <= daily_df["S2"]
daily_df['profitR5'] = daily_df['low'] <= daily_df["R1"] - daily_df['R3'] + daily_df["R1"] - 0.0001*40
daily_df['profitS5'] = daily_df['high'] >= daily_df["S1"] - daily_df["S3"] + daily_df["S1"] + 0.0001*40
daily_df

,time,open,high,low,close,pivot,prange,R1,R2,R3,...,sma_11,prev_sma_11,sma_114,stopeR4,bearish_pivot,stopeS4,bullish_pivot,profitR5,profitS5,entring_stoploss_profit
0,2024-01-02,1.10424,1.10443,1.09383,1.09413,1.097463,0.01060,1.101513,1.104014,1.108063,...,1.09413,NaN,NaN,False,True,False,False,False,True,False
1,2024-01-03,1.09423,1.09654,1.08929,1.09196,1.092597,0.00725,1.095366,1.097077,1.099847,...,1.09196,1.09413,NaN,False,False,False,False,False,False,False
2,2024-01-04,1.09163,1.09721,1.09140,1.09478,1.094463,0.00581,1.096683,1.098054,1.100273,...,1.09478,1.09196,NaN,False,False,False,False,False,False,False
3,2024-01-05,1.09442,1.09984,1.08765,1.09409,1.093860,0.01219,1.098517,1.101393,1.106050,...,1.09409,1.09478,NaN,False,False,False,False,False,False,False
4,2024-01-08,1.09411,1.09789,1.09224,1.09508,1.095070,0.00565,1.097228,1.098562,1.100720,...,1.09508,1.09409,NaN,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73,2024-04-12,1.07271,1.07293,1.06225,1.06413,1.066437,0.01068,1.070516,1.073037,1.077117,...,1.06413,1.07258,1.079191,False,False,False,False,False,False,False
74,2024-04-15,1.06342,1.06650,1.06202,1.06225,1.063590,0.00448,1.065301,1.066359,1.068070,...,1.06225,1.06413,1.077699,False,True,False,False,False,False,False
75,2024-04-16,1.06236,1.06536,1.06011,1.06184,1.062437,0.00525,1.064442,1.065681,1.067687,...,1.06184,1.06225,1.076203,False,False,False,False,False,False,False
76,2024-04-17,1.06179,1.06796,1.06060,1.06733,1.065297,0.00736,1.068108,1.069845,1.072657,...,1.06733,1.06184,1.075392,False,False,False,True,False,False,False


In [55]:
def entring_stoploss_profit (row) :
    bullish_crossover = row['sma_1'] > row['sma_14'] and row['sma_11'] > row['sma_114'] and row['prev_sma_1'] < row['sma_14'] and row['prev_sma_11'] < row['sma_114']
    bearish_crossover = row['sma_1'] < row['sma_14'] and row['sma_11'] < row['sma_114'] and row['prev_sma_1'] > row['sma_14'] and row['prev_sma_11'] > row['sma_114']
    buy_signals = bullish_crossover and row["bullish_pivot"] and open_positions and first_trade_per_day 
    sell_signals = bearish_crossover and row["bearish_pivot"] and open_positions and first_trade_per_day
    close_condition1 = row["stopeS4"] or end_trading_time
    close_condition2 = row["stopeR4"] or end_trading_time
    if buy_signals :
        return True
    
    elif sell_signals :
        return True
        
    else :
        return False

ohlc_df['entring_stoploss_profit'] = ohlc_df.apply(entring_stoploss_profit, axis=1)
ohlc_df

,time,open,high,low,close,R1,R2,R3,S1,S2,...,sma_11,prev_sma_11,sma_114,stopeR4,bearish_pivot,stopeS4,bullish_pivot,profitR5,profitS5,entring_stoploss_profit
0,2024-01-02 00:00:00,1.10424,1.10434,1.10416,1.10430,1.101513,1.104014,1.108063,1.093414,1.090913,...,1.09413,NaN,NaN,False,True,False,False,False,True,False
1,2024-01-02 00:30:00,1.10422,1.10443,1.10416,1.10434,1.095366,1.097077,1.099847,1.089827,1.088116,...,1.09196,1.09413,NaN,True,True,False,False,False,True,False
2,2024-01-02 01:00:00,1.10438,1.10442,1.10369,1.10375,1.096683,1.098054,1.100273,1.092244,1.090873,...,1.09478,1.09196,NaN,True,True,False,False,False,True,False
3,2024-01-02 01:30:00,1.10375,1.10384,1.10355,1.10364,1.098517,1.101393,1.106050,1.089203,1.086327,...,1.09409,1.09478,NaN,False,True,False,False,False,True,False
4,2024-01-02 02:00:00,1.10366,1.10380,1.10341,1.10345,1.097228,1.098562,1.100720,1.092912,1.091578,...,1.09508,1.09409,NaN,True,True,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3736,2024-04-18 21:00:00,1.06481,1.06493,1.06436,1.06437,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,False,False,False,False,False,False,False
3737,2024-04-18 21:30:00,1.06437,1.06485,1.06419,1.06451,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,False,False,False,False,False,False,False
3738,2024-04-18 22:00:00,1.06450,1.06482,1.06424,1.06433,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,False,False,False,False,False,False,False
3739,2024-04-18 22:30:00,1.06432,1.06468,1.06431,1.06437,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,False,False,False,False,False,False,False


In [73]:
# visualizing the ohlc data
fig = go.Figure(data = [go.Candlestick(x = ohlc_df['time'],
                                       open = ohlc_df['open'],
                                       high = ohlc_df['high'],
                                       low = ohlc_df['low'],
                                       close = ohlc_df['close'])])

fig.update_layout(height=600, xaxis_rangeslider_visible=False)
fig.add_trace(go.Scatter(
    x= ohlc_df['time'],
    y= daily_df["R2"],
    marker = dict(color = "orange", size = 2),
    name = "R2"
))
fig

KeyError: 'R2'